In [1]:
#import dask.dataframe as dd
import pandas as pd
import numpy as np
import dask.dataframe as dd
# Set up Dask to use multiple cores
from dask.distributed import Client,LocalCluster

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import plotly.graph_objects as go
import plotly.io as pio
# Set default renderer
pio.renderers.default = 'iframe'  #  See [1]

# Set up the local cluster
cluster = LocalCluster(
    n_workers=20,  # Use one worker per core
    threads_per_worker=1,  # Single-threaded workers for CPU-bound tasks
    memory_limit='20GB'  # Allocate memory per worker
)

# Create the client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 372.53 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43097,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 372.53 GiB
Comm: tcp://127.0.0.1:33637,Total threads: 1
Dashboard: http://127.0.0.1:40779/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:39239,


2024-10-17 16:45:05,764 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,776 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,823 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,825 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,832 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,833 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,846 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,850 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,853 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,860 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,862 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,866 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,869 - distributed.nanny - WARNING - Restarting worker
2024-10-17 16:45:05,875 - distributed.

# PA matrix

## Sample statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")  # Increased blocksize due to large memory

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('pa_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

## Unitig statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")  # Increased blocksize due to large memory

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('pa_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

## Hypothesis tests

In [3]:
metadata=pd.read_csv("../metadata/Damselflies_metadata_with_morph.csv",index_col=0)
# Load column names from fof_filtered_pa.txt
with open('../metadata/fof_filtered_pa.txt', 'r') as f:
    column_names = [line.strip() for line in f]
column_names = [name.replace("data/unitigs/", "").replace(".unitigs.fa.gz", "") for name in column_names]


In [11]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

# Create a dictionary mapping sample names to their morphs
sample_to_morph = dict(zip(metadata['Run'], metadata['Morph']))  # adjust 'SampleID' if it's named differently

# Create dictionaries for each morph group
morph_groups = {morph: [] for morph in metadata['Morph'].unique()}

for sample in column_names:
    if sample in sample_to_morph:
        morph = sample_to_morph[sample]
        morph_groups[morph].append(sample)

# Print group sizes for verification
for morph, samples in morph_groups.items():
    print(f"Group {morph}: {len(samples)} samples")

# Load your data, specifying the column names
df = dd.read_csv('../unitig_matrix_pa/unitigs.filter.02.csv', header=None, names=['unitig_id'] + column_names)

def epsilon_squared(h_statistic, n):
    return h_statistic / (n**2 - 1) / (n + 1)

def compute_test(row):
    group_data = {morph: row[samples].values for morph, samples in morph_groups.items()}
    
    # Check if all groups have variation
    if all(len(set(group)) > 1 for group in group_data.values()):
        # Perform Kruskal-Wallis H-test
        h_statistic, p_value = stats.kruskal(*group_data.values())
        n = sum(len(group) for group in group_data.values())
        effect_size = epsilon_squared(h_statistic, n)
    else:
        # If any group has no variation, set h_statistic and effect_size to 0 and p_value to 1
        h_statistic, p_value, effect_size = 0, 1, 0
    
    return pd.Series({'h_statistic': h_statistic, 'p_value': p_value, 'effect_size': effect_size})

# Define the metadata for Dask
meta = pd.DataFrame(columns=['h_statistic', 'p_value', 'effect_size']).astype({'h_statistic': 'float64', 'p_value': 'float64', 'effect_size': 'float64'})

# Apply the test to each chunk
results = df.map_partitions(lambda chunk: chunk.apply(compute_test, axis=1), meta=meta).compute()

# Perform Benjamini-Hochberg correction
_, adjusted_p_values, _, _ = multipletests(results['p_value'], method='fdr_bh')

# Add adjusted p-values to the results
results['adjusted_p_value'] = adjusted_p_values

# Calculate the number of significant results
alpha = 0.05  # Set your desired significance level
significant_count = sum(results['adjusted_p_value'] < alpha)

print(f"Number of tests performed: {len(results)}")
print(f"Number of significant results at α = {alpha}: {significant_count}")
print(f"Proportion of significant results: {significant_count / len(results):.2%}")

# Optional: Save results to file
results.to_csv('kruskal_wallis_results_pa_matrix.csv', index=False)

# Display summary statistics
print("\nSummary Statistics:")
print(results.describe())

# Optional: Display top significant results
print("\nTop 10 significant unitigs:")
print(results.sort_values('adjusted_p_value').head(10))

Group O: 20 samples
Group A: 18 samples
Group nan: 7 samples
Group I: 19 samples


/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


Number of tests performed: 18005058
Number of significant results at α = 0.05: 39524
Proportion of significant results: 0.22%

Summary Statistics:
        h_statistic       p_value   effect_size  adjusted_p_value
count  1.800506e+07  1.800506e+07  1.800506e+07      1.800506e+07
mean   2.864268e+00  5.803036e-01  1.076085e-05      9.140086e-01
std    3.428268e+00  3.616419e-01  1.287975e-05      1.874796e-01
min    0.000000e+00  3.993292e-12  0.000000e+00      2.007844e-05
25%    0.000000e+00  2.361886e-01  0.000000e+00      9.446495e-01
50%    1.888787e+00  5.958070e-01  7.096035e-06      1.000000e+00
75%    4.245065e+00  1.000000e+00  1.594840e-05      1.000000e+00
max    5.610316e+01  1.000000e+00  2.107755e-04      1.000000e+00

Top 10 significant unitigs:
        h_statistic       p_value  effect_size  adjusted_p_value
146497    54.012569  1.115155e-11     0.000203           0.00002
180624    54.012569  1.115155e-11     0.000203           0.00002
370011    56.103161  3.993292e-12  

In [ ]:
results.shape

# Abundance matrix

## Sample statistics

In [ ]:
import dask.dataframe as dd

#client = Client(n_workers=20, memory_limit='40GB')

df = dd.read_csv("../unitig_matrix_abundance/A.mat",sep=" ")

# Set the first column as the index
df = df.set_index(df.columns[0])

# Calculate statistics for each column
stats = df.describe().compute()

# Calculate additional quantiles
quantiles = df.quantile([0.25, 0.75]).compute()

# Combine the statistics and quantiles
combined_stats = stats.transpose()
combined_stats['25%'] = quantiles.loc[0.25]
combined_stats['75%'] = quantiles.loc[0.75]

# Reorder columns
column_order = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
combined_stats = combined_stats[column_order]

# Print the combined statistics
print(combined_stats)

# Save the results to a CSV file
combined_stats.to_csv('abundance_unitig_matrix_02_filtered_col_statistics.csv')


## Unitig statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_abundance/A.mat", 
                 sep='\s+',
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="100MB")  # Adjust this value as needed

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('abundance_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

In [4]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
# Create a dictionary mapping sample names to their morphs
sample_to_morph = dict(zip(metadata['Run'], metadata['Morph']))  # adjust 'Run' if it's named differently

# Create dictionaries for each morph group
morph_groups = {morph: [] for morph in metadata['Morph'].unique()}

for sample in column_names:
    if sample in sample_to_morph:
        morph = sample_to_morph[sample]
        morph_groups[morph].append(sample)

# Print group sizes for verification
for morph, samples in morph_groups.items():
    print(f"Group {morph}: {len(samples)} samples")

# Load your data, specifying the column names
df = dd.read_csv("../unitig_matrix_abundance/A.mat", 
                 sep='\s+',
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="100MB",  # Adjust this value as needed
                 header=None, 
                 names=['unitig_id'] + column_names)

def epsilon_squared(h_statistic, n):
    return h_statistic / ((n**2 - 1) / (n + 1))

def compute_test(row):
    group_data = {morph: row[samples].values for morph, samples in morph_groups.items()}
    
    if all(len(set(group)) > 1 for group in group_data.values()):
        h_statistic, p_value = stats.kruskal(*group_data.values())
        n = sum(len(group) for group in group_data.values())
        effect_size = epsilon_squared(h_statistic, n)
    else:
        h_statistic, p_value, effect_size = 0, 1, 0
    
    return pd.Series({'h_statistic': h_statistic, 'p_value': p_value, 'effect_size': effect_size})

# Define the metadata for Dask
meta = pd.DataFrame(columns=['h_statistic', 'p_value', 'effect_size']).astype({'h_statistic': 'float64', 'p_value': 'float64', 'effect_size': 'float64'})

# Apply the test to each chunk
results = df.map_partitions(lambda chunk: chunk.apply(compute_test, axis=1), meta=meta).compute()

# Perform Benjamini-Hochberg correction
_, adjusted_p_values, _, _ = multipletests(results['p_value'], method='fdr_bh')

# Add adjusted p-values to the results
results['adjusted_p_value'] = adjusted_p_values

# Calculate the number of significant results
alpha = 0.05  # Set your desired significance level
significant_count = sum(results['adjusted_p_value'] < alpha)

print(f"Number of tests performed: {len(results)}")
print(f"Number of significant results at α = {alpha}: {significant_count}")
print(f"Proportion of significant results: {significant_count / len(results):.2%}")

# Save results to file
results.to_csv('kruskal_wallis_results_abundance_matrix.csv', index=False)

# Display summary statistics
print("\nSummary Statistics:")
print(results.describe())

# Display top significant results
print("\nTop 10 significant unitigs:")
print(results.sort_values('adjusted_p_value').head(10))

# If you're using Dask distributed, uncomment the following line:
# client.close()

Group O: 20 samples
Group A: 18 samples
Group nan: 7 samples
Group I: 19 samples
Number of tests performed: 5285543
Number of significant results at α = 0.05: 152
Proportion of significant results: 0.00%

Summary Statistics:
        h_statistic       p_value   effect_size  adjusted_p_value
count  5.285543e+06  5.285543e+06  5.285543e+06      5.285543e+06
mean   1.525852e+00  7.564675e-01  2.421987e-02      9.985426e-01
std    2.521741e+00  3.321119e-01  4.002763e-02      2.680637e-02
min    0.000000e+00  8.170600e-11  0.000000e+00      4.318606e-04
25%    0.000000e+00  4.958598e-01  0.000000e+00      1.000000e+00
50%    0.000000e+00  1.000000e+00  0.000000e+00      1.000000e+00
75%    2.388067e+00  1.000000e+00  3.790582e-02      1.000000e+00
max    4.995421e+01  1.000000e+00  7.929240e-01      1.000000e+00

Top 10 significant unitigs:
        h_statistic       p_value  effect_size  adjusted_p_value
218407    49.954210  8.170600e-11     0.792924          0.000432
25945     45.235223  8

In [10]:
results[results["adjusted_p_value"]<0.05].describe()

,h_statistic,p_value,effect_size,adjusted_p_value
count,152.000000,1.520000e+02,152.000000,152.000000
mean,33.340851,5.498634e-07,0.529220,0.029597
std,3.563084,4.539242e-07,0.056557,0.013450
min,29.919924,8.170600e-11,0.474919,0.000432
25%,30.971410,1.861325e-07,0.491610,0.023645
50%,32.371480,4.370668e-07,0.513833,0.028086
75%,34.128391,8.619373e-07,0.541720,0.039875
max,49.954210,1.434631e-06,0.792924,0.049887


# PCA pa 

In [2]:
import dask.dataframe as dd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from dask.distributed import Client
import pandas as pd

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")

# Separate the index column if it exists
if 'unitig_id' in df.columns:
    index_column = df['unitig_id']
    df = df.drop('unitig_id', axis=1)
else:
    index_column = df.index

# Transpose the dataframe
df_transposed = df.compute().T

# Initialize IncrementalPCA
ipca = IncrementalPCA(n_components=2, batch_size=1000)

# Fit and transform the transposed data
pca_result = ipca.fit_transform(df_transposed)

# Create a new dataframe with the PCA results
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'], index=df_transposed.index)

# Get the explained variance ratio
explained_variance_ratio = ipca.explained_variance_ratio_

# Save the PCA result
pca_df.to_csv('pca_result_rows.csv')

# Save the explained variance ratio
np.savetxt('explained_variance_ratio_rows.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings (optional, but can be useful for interpretation)
component_loadings = pd.DataFrame(ipca.components_.T, index=index_column, columns=['PC1', 'PC2'])
component_loadings.to_csv('component_loadings_rows.csv')

print("PCA completed.")
print(f"Explained variance ratio: {explained_variance_ratio}")
print("Results saved to 'pca_result_rows.csv', 'explained_variance_ratio_rows.csv', and 'component_loadings_rows.csv'")




2024-10-17 16:13:08,317 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 15.56 GiB -- Worker memory limit: 18.63 GiB
2024-10-17 16:13:12,066 - distributed.worker.memory - WARNING - Worker is at 60% memory usage. Resuming worker. Process memory: 11.33 GiB -- Worker memory limit: 18.63 GiB
2024-10-17 16:13:12,068 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 11.33 GiB -- Worker memory limit: 18.63 GiB


PCA completed.
Explained variance ratio: [9.99696744e-01 1.49520954e-06]
Results saved to 'pca_result_rows.csv', 'explained_variance_ratio_rows.csv', and 'component_loadings_rows.csv'


# iPCA on abundance

In [3]:
import dask.dataframe as dd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from dask.distributed import Client
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Read the CSV file
df = dd.read_csv("../unitig_matrix_abundance/A.mat", 
                 sep='\s+',
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="100MB")  # Adjust this value as needed

# Separate the index column if it exists
if df.columns[0] != '0':  # Assuming the first column is not numeric
    index_column = df[df.columns[0]]
    df = df.drop(df.columns[0], axis=1)
else:
    index_column = df.index

# Compute the dataframe and transpose it
df_transposed = df.compute().T

# Standardize the data
scaler = StandardScaler()
df_transposed_scaled = scaler.fit_transform(df_transposed)

# Initialize IncrementalPCA
ipca = IncrementalPCA(n_components=2, batch_size=1000)

# Fit and transform the transposed and scaled data
pca_result = ipca.fit_transform(df_transposed_scaled)

# Create a new dataframe with the PCA results
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'], index=df_transposed.index)

# Get the explained variance ratio
explained_variance_ratio = ipca.explained_variance_ratio_

# Save the PCA result
pca_df.to_csv('pca_result_abundance_rows.csv')

# Save the explained variance ratio
np.savetxt('explained_variance_ratio_abundance_rows.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings (optional, but can be useful for interpretation)
component_loadings = pd.DataFrame(ipca.components_.T, index=index_column, columns=['PC1', 'PC2'])
component_loadings.to_csv('component_loadings_abundance_rows.csv')

print("PCA completed.")
print(f"Explained variance ratio: {explained_variance_ratio}")
print("Results saved to 'pca_result_abundance_rows.csv', 'explained_variance_ratio_abundance_rows.csv', and 'component_loadings_abundance_rows.csv'")



PCA completed.
Explained variance ratio: [0.04267896 0.02676159]
Results saved to 'pca_result_abundance_rows.csv', 'explained_variance_ratio_abundance_rows.csv', and 'component_loadings_abundance_rows.csv'


# PCA on significant unitigs pa

In [10]:
import dask.dataframe as dd
import pandas as pd

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")

# Read the hypothesis test results
hypothesis_test_pa = pd.read_csv("kruskal_wallis_results_pa_matrix.csv")

# Filter the hypothesis test results to get significant unitigs
significant_unitigs = hypothesis_test_pa[hypothesis_test_pa['adjusted_p_value'] < 0.05]

# Get the list of column names (assuming the first column is the unitig ID)
unitig_column = df.columns[0]

# Create a set of significant unitig IDs for faster lookup
significant_unitig_ids = set(significant_unitigs.index)

# Define a function to filter rows
def filter_significant(row):
    return row[unitig_column] in significant_unitig_ids

# Apply the filter function
df_filtered = df[df.map_partitions(lambda df: df.apply(filter_significant, axis=1))]

# Compute the result (this might take some time depending on the size of your data)
df_filtered = df_filtered.compute()

# Save the filtered dataframe to a new CSV file
df_filtered.to_csv("filtered_unitigs_pa.csv", index=False)

# Print the shape of the filtered dataframe
print(f"Shape of filtered dataframe: {df_filtered.shape}")

Shape of filtered dataframe: (0, 65)


In [25]:
# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")

# Read the hypothesis test results
hypothesis_test_pa = dd.read_csv("kruskal_wallis_results_pa_matrix.csv")

hypothesis_test_pa['Unitigs_id'] = df['Unitigs_id']

# Save the updated hypothesis test results
hypothesis_test_pa.to_csv("kruskal_wallis_results_pa_matrix_updated.csv",single_file=True)

2024-10-17 16:47:15,885 - distributed.worker - ERROR - Compute Failed
Key:       ('getitem-assign-09dd50c23d8101daeb26170f0929fac7', 6)
State:     executing
Function:  execute_task
args:      ((<function Fused._execute_task at 0x7fb36c17e5c0>, {'getitem-assign-09dd50c23d8101daeb26170f0929fac7': ('assign-9786dc5736ccb91feb835be50921dbec', 6), ('assign-9786dc5736ccb91feb835be50921dbec', 6): (<function assign at 0x7fb36c16b240>,          h_statistic   p_value  effect_size  adjusted_p_value
34          0.998092  0.801714     0.000004          1.000000
65          0.000000  1.000000     0.000000          1.000000
73          3.452922  0.326919     0.000013          1.000000
80          1.481444  0.686560     0.000006          1.000000
97          0.000000  1.000000     0.000000          1.000000
...              ...       ...          ...               ...
1200798     0.000000  1.000000     0.000000          1.000000
1200840     8.485897  0.036968     0.000032          0.539817
1200842     

ValueError: cannot reindex on an axis with duplicate labels

In [28]:
def check_index_uniqueness(ddf):
    return ddf.index.nunique().compute() == len(ddf)

print("df index is unique:", check_index_uniqueness(df))
print("hypothesis_test_pa index is unique:", check_index_uniqueness(hypothesis_test_pa))

df index is unique: False


2024-10-17 16:54:32,743 - distributed.worker - ERROR - Compute Failed
Key:       ('getitem-fused-operation-eda8bd063b1d0ac488061e8944072c02', 3)
State:     executing
Function:  execute_task
args:      ((<function Fused._execute_task at 0x7f9410c64f40>, {'getitem-fused-operation-eda8bd063b1d0ac488061e8944072c02': ('operation-50b637f7da376aeac097bcb809d48d93', 3), ('operation-50b637f7da376aeac097bcb809d48d93', 3): (<function AssignPartitioningIndex.operation at 0x7f94104974c0>, ('to_frame-4bf9084b2f7b35326a0e90a1b60baa2f', 3), ['__index__'], '_partitions', 15, Empty DataFrame
Columns: [__index__]
Index: [], None), ('to_frame-4bf9084b2f7b35326a0e90a1b60baa2f', 3): (<function apply at 0x7f943c9b44a0>, <methodcaller: to_frame>, [('drop_duplicates-b4acceee0f533e6305730a3fc1964cf1', 3)], {'name': '__index__'}), ('drop_duplicates-b4acceee0f533e6305730a3fc1964cf1', 3): (<function apply at 0x7f943c9b44a0>, <methodcaller: drop_duplicates>, [('getattr-cfc882573bfebe4ddb4b8a9ac7f33e69', 3)], {'keep

ValueError: cannot reindex on an axis with duplicate labels

s/Damselflies/lib/python3.11/site-packages/dask_expr/_expr.py", line 1846, in assign\n    df[name] = val\n    ~~^^^^^^\n  File "/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/pandas/core/frame.py", line 4311, in __setitem__\n    self._set_item(key, value)\n  File "/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/pandas/core/frame.py", line 4524, in _set_item\n    value, refs = self._sanitize_column(value)\n                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/pandas/core/frame.py", line 5263, in _sanitize_column\n    return _reindex_for_setitem(value, self.index)\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/pandas/core/frame.py", line 12692, in _reindex_for_setitem\n    raise err\n  File "/pasteur/appa/homes/cduitama/venvs/Damselflies/lib/python3.11/site-packages/